# load models and media

In [1]:
# # define the environment path 
# import os
# local_cobrakbase_path = os.path.join("C:", "Users", 'Andrew Freiburger',"Documents","Argonne","cobrakbase")
# os.environ["HOME"] = local_cobrakbase_path

# # import the KBase
# import cobrakbase
# with open("/Users/andrewfreiburger/.kbase/token") as token_file:
# # with open("C:/Users/Andrew Freiburger/Documents/Argonne/kbase_token.txt") as token_file:
#     kbase_api = cobrakbase.KBaseAPI(token_file.readline().strip())

/Users/andrewfreiburger/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


modelseedpy 0.3.3
cobrakbase 0.4.0


In [1]:
# models = [kbase_api.get_from_ws(mdl, 175377) for mdl in ["Rhizobium_sp._CF142_2510917030.fna.RAST.mdl", "Pantoea_sp._YR343_2511231025.fna.RAST.mdl",
#                                                          "Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl", "Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl"]
#                                                          ] if "models" not in locals() else models
# from cobra.io import write_sbml_model
# print(models)
# for model in models:
#     write_sbml_model(model, f"{model.id}.xml")

# from modelseedpy import FBAHelper
# print(dir(r2a))
# print(r2a.data)

# r2a = kbase_api.get_from_ws("R2A_edition_003", 175377)
# from json import dump
# with open("R2A_media.json", 'w') as jsonOut:
#     dump(FBAHelper.convert_kbase_media(r2a), jsonOut, indent=3)

from cobra.io import read_sbml_model
from json import load
from glob import glob

models = []
for path in glob("*.xml"):
    model = read_sbml_model(path)
    model.id = path.replace(".xml", '')
    models.append(model)
media = load(open("R2A_media.json", 'r'))

'' is not a valid SBML 'SId'.
'' is not a valid SBML 'SId'.
'' is not a valid SBML 'SId'.
'' is not a valid SBML 'SId'.


# old 2024 code

In [15]:
from cobra import Model, Metabolite, Reaction
from modelseedpy import MSModelUtil
import re

def parse_id(object):
    if re.search("(.+)_([a-z]+)(\d*)$", object.id) != None:
        m = re.search("(.+)_([a-z]+)(\d*)$", object.id)
        return (m[1], m[2], m[3])
    return None

def build_from_species_models2(
        models, mdlid=None, name=None, names=[], abundances=None,basemodel=None
    ):
        """Merges the input list of single species metabolic models into a community metabolic model

        Parameters
        ----------
        models : list<Cobra.Model>
            List of models to be merged into a community model
        mdlid : string
            String specifying community model ID
        name : string
            String specifying community model name
        names : list<string>
            List of human readable names for models being merged
        abundances : dict<string,float>
            Hash of relative abundances for input models in community model

        Returns
        -------
        Cobra.Model
            Community model object

        Raises
        ------
        """
        if basemodel:
            newmodel = basemodel
        else:
            newmodel = Model(mdlid, name)
        newutl = MSModelUtil.get(newmodel)
        biomass_compounds = []
        index = 1
        biomass_index = 2
        for model in models:
            new_metabolites = []
            new_reactions = []
            # Rename metabolites
            for met in model.metabolites:
                # Renaming compartments
                if re.search("[a-z+](\d*)$", met.compartment):
                    m = re.search("([a-z]+)(\d*)$", met.compartment)
                    if len(m[2]) == 0:
                        if m[1] == "e":
                            met.compartment += "0"
                        else:
                            met.compartment += str(index)
                    elif m[1] == "e":
                        met.compartment = m[1] + "0"
                    else:
                        met.compartment = m[1] + str(index)
                # Processing metabolite ID
                output = parse_id(met)
                if output == None:
                    if met.compartment[0] != "e":
                        met.id += str(index)
                elif output[1] != "e":
                    if len(output[2]) == 0:
                        met.id = met.id + str(index)
                    else:
                        met.id = output[0] + "_" + output[1] + str(index)
                if met.id not in newmodel.metabolites:
                    new_metabolites.append(met)
                    if newutl.metabolite_msid(met) == "cpd11416":
                        biomass_compounds.append(met)
            # Rename reactions
            for rxn in model.reactions:
                if rxn.id[0:3] != "EX_":
                    if re.search("^(bio)(\d+)$", rxn.id) != None:
                        rxn.id = "bio" + str(biomass_index)
                        biomass_index += 1
                    else:
                        output = parse_id(rxn)
                        if output == None:
                            if rxn.compartment.id[0] != "e":
                                rxn.id += str(index)
                        elif output[1] != "e":
                            if len(output[2]) == 0:
                                rxn.id = rxn.id + str(index)
                            else:
                                rxn.id = output[0] + "_" + output[1] + str(index)
                if rxn.id not in newmodel.reactions:
                    new_reactions.append(rxn)
            # Adding new reactions and compounds to base model
            newmodel.add_reactions(new_reactions)
            newmodel.add_metabolites(new_metabolites)
            index += 1
        # Create community biomass
        comm_biomass = Metabolite("cpd11416_c0", None, "Community biomass", 0, "c0")
        metabolites = {comm_biomass: 1}
        comm_biorxn = Reaction(id="bio1", name="bio1", lower_bound=0, upper_bound=100)
        count = len(biomass_compounds)
        for cpd in biomass_compounds:
            metabolites[cpd] = -1 / count
        comm_biorxn.add_metabolites(metabolites)
        newmodel.add_reactions([comm_biorxn])
        newutl.add_exchanges_for_metabolites([comm_biomass], 0, 100, "SK_")
        return newmodel

example_pairwise_chris = build_from_species_models2(models[:2])

In [16]:
print(example_pairwise_chris.reactions.bio1.reaction)
sol = example_pairwise_chris.optimize()
print({"bio2": sol.fluxes["bio2"], "bio3": sol.fluxes["bio3"]})

0.5 cpd11416_c1 + 0.5 cpd11416_c2 --> cpd11416_c0
{'bio2': 0.0, 'bio3': 0.0}


In [17]:
example_pairwise = build_from_species_models(models[:2])

The ID rxn02201_c1 is changed with  to create the final ID rxn02201_c1
The ID rxn00351_c1 is changed with  to create the final ID rxn00351_c1
The ID rxn07431_c1 is changed with  to create the final ID rxn07431_c1
The ID rxn00836_c1 is changed with  to create the final ID rxn00836_c1
The ID rxn00423_c1 is changed with  to create the final ID rxn00423_c1
The ID rxn00364_c1 is changed with  to create the final ID rxn00364_c1
The ID rxn05561_c1 is changed with  to create the final ID rxn05561_c1
The ID rxn03408_c1 is changed with  to create the final ID rxn03408_c1
The ID rxn02177_c1 is changed with  to create the final ID rxn02177_c1
The ID rxn05250_c1 is changed with  to create the final ID rxn05250_c1
The ID rxn00646_c1 is changed with  to create the final ID rxn00646_c1
The ID rxn05440_c1 is changed with  to create the final ID rxn05440_c1
The ID rxn01673_c1 is changed with  to create the final ID rxn01673_c1
The ID rxn00247_c1 is changed with  to create the final ID rxn00247_c1
The ID

The ID rxn02201_c2 is changed with  to create the final ID rxn02201_c2
The ID rxn00351_c2 is changed with  to create the final ID rxn00351_c2
The ID rxn00836_c2 is changed with  to create the final ID rxn00836_c2
The ID rxn02209_c2 is changed with  to create the final ID rxn02209_c2
The ID rxn01094_c2 is changed with  to create the final ID rxn01094_c2
The ID rxn00423_c2 is changed with  to create the final ID rxn00423_c2
The ID rxn00364_c2 is changed with  to create the final ID rxn00364_c2
The ID rxn05561_c2 is changed with  to create the final ID rxn05561_c2
The ID rxn03408_c2 is changed with  to create the final ID rxn03408_c2
The ID rxn02177_c2 is changed with  to create the final ID rxn02177_c2
The ID rxn05250_c2 is changed with  to create the final ID rxn05250_c2
The ID rxn00646_c2 is changed with  to create the final ID rxn00646_c2
The ID rxn05440_c2 is changed with  to create the final ID rxn05440_c2
The ID rxn00935_c2 is changed with  to create the final ID rxn00935_c2
The ID

In [18]:
print(example_pairwise.reactions.bio1.reaction)
sol = example_pairwise.optimize()
print({"bio2": sol.fluxes["bio2"], "bio3": sol.fluxes["bio3"]})

0.5 cpd11416_c1 + 0.5 cpd11416_c2 --> cpd11416_c0
{'bio2': 58.34675810455965, 'bio3': 58.346758104559655}


In [4]:
from pandas import set_option
set_option("display.max_columns", None)
display(fluxes)
print(mets)

,model1,model2,media,monoculture growth model1,monoculture growth model2,coculture growth model1,coculture growth model2,community growth,MRO_model1,MRO_model2,CIP,MIP_model1 (costless),MIP_model2 (costless),BSS_model1,BSS_model2,PC_model1,PC_model2,PC_comm,BIT,GYD1,GYD2,FS
0,Rhizobium_sp._CF142_2510917030.fna.RAST.mdl,Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl,R2A_edition_003,25.758,41.63,0.882274,0.117726,53.532,64.516% (20/31),44.444% (20/45),19,3 (2),2 (0),73.333%,51.613%,2.2652,1.4016,1.7317,mutualism,0.61619,0.38126,0.3385
1,Rhizobium_sp._CF142_2510917030.fna.RAST.mdl,Streptomyces_mirabilis_YR139_2582581314.fna.RA...,R2A_edition_003,25.758,57.961,0.605618,0.394382,72.17,58.065% (18/31),56.250% (18/32),15,0 (0),3 (1),50.0%,51.613%,2.054,0.91281,1.2639,commensalism,1.2502,0.5556,0.26929
2,Rhizobium_sp._CF142_2510917030.fna.RAST.mdl,Pantoea_sp._YR343_2511231025.fna.RAST.mdl,R2A_edition_003,25.758,74.195,0.765925,0.234075,69.84,64.516% (20/31),50.000% (20/40),15,2 (0),1 (1),62.5%,51.613%,2.2652,0.7864,1.1675,parasitism,1.8805,0.65283,0.31432
3,Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl,Streptomyces_mirabilis_YR139_2582581314.fna.RA...,R2A_edition_003,41.63,57.961,0.55973,0.44027,99.79,40.000% (18/45),56.250% (18/32),21,3 (1),4 (1),46.875%,77.778%,1.565,1.1241,1.3084,mutualism,0.3923,0.28176,0.24154
4,Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl,Pantoea_sp._YR343_2511231025.fna.RAST.mdl,R2A_edition_003,41.63,74.195,0.675548,0.324452,112.32,62.222% (28/45),70.000% (28/40),21,6 (1),7 (2),65.0%,71.111%,1.8661,1.0471,1.3414,commensalism,0.78226,0.43891,0.36201
5,Streptomyces_mirabilis_YR139_2582581314.fna.RA...,Pantoea_sp._YR343_2511231025.fna.RAST.mdl,R2A_edition_003,57.961,74.195,0.694389,0.305611,105.65,65.625% (21/32),52.500% (21/40),17,5 (1),5 (3),70.0%,50.0%,1.1467,0.8958,1.0058,parasitism,0.28008,0.2188,0.21656


[{'MRO metabolites': ['cpd00254', 'cpd00080', 'cpd00030', 'cpd00058', 'cpd00264', 'cpd00071', 'cpd00023', 'cpd00099', 'cpd00149', 'cpd00034', 'cpd00051', 'cpd00268', 'cpd00039', 'cpd00007', 'cpd00205', 'cpd00119', 'cpd00028', 'cpd00063', 'cpd10516', 'cpd00048'], 'CIP metabolites': ['cpd00418', 'cpd00020', 'cpd15378', 'cpd00117', 'cpd00067', 'cpd00036', 'cpd00047', 'cpd00073', 'cpd00129', 'cpd00001', 'cpd00221', 'cpd00041', 'cpd01981', 'cpd00159', 'cpd00309', 'cpd00029', 'cpd00013', 'cpd00011', 'cpd00180'], 'MIP model1 metabolites': ['cpd00013', 'cpd00048', 'cpd00239'], 'MIP model2 metabolites': ['cpd01080', 'cpd00106'], 'BSS model1 metabolites': {'cpd28083', 'cpd15725', 'cpd02439', 'cpd00493', 'cpd21481', 'cpd00672', 'cpd03126', 'cpd03646', 'cpd00508', 'cpd21288', 'cpd27021', 'cpd01914', 'cpd03897', 'cpd00190', 'cpd11559', 'cpd00036', 'cpd11310', 'cpd00076', 'cpd11528', 'cpd00179', 'cpd01977', 'cpd15697', 'cpd02724', 'cpd00018', 'cpd00037', 'cpd11571', 'cpd03444', 'cpd02566', 'cpd02700

# uploading the kChip genomes

In [34]:
from pandas import read_csv, DataFrame
from numpy import nan
from glob import glob
from os import path

seenFASTQ = []
rows = []
for fastq in glob("/Users/afreiburger/Documents/CommScores/examples/240412_Gupta_Hanke_fastqs/*"):
    baseName = path.basename(fastq)
    name = "_".join(baseName.split("_")[:2])
    if name in seenFASTQ:  print(name) ;  continue
    if "_R1_" in fastq:
        left = f"240412_Gupta_Hanke_fastqs/{baseName}.gz"
        right = f"240412_Gupta_Hanke_fastqs/{baseName.replace("Users", "cobrakbase")}.gz"
    elif "_R2_" in fastq:
        right = f"240412_Gupta_Hanke_fastqs/{baseName}.gz"
        left = f"240412_Gupta_Hanke_fastqs/{baseName.replace("_R1_", "_R2_")}.gz"
    seqTech = "Illumina"
    singleGenome = "1"
    read_orientation_output = "0"
    seenFASTQ.append(name)
    # print(seenFASTQ)
    rows.append([left, right, seqTech, name, singleGenome, read_orientation_output, nan, nan])
    # print("\t\t".join([left, right, name, singleGenome, read_orientation_output]))

PAg3_S12
ADP1_S21
SF1_S9
Ecoli_S20
CF_S4
PP_S15
PAg2_S8
LA_S10
PAg1_S5
RP1_S2
KA_S6
PH_S13
PR1_S14
PR2_S17
EA_S1
EL_S16
PAl_S11
BI_S3
PK_S18
PAr_S19
RP2_S7


In [40]:
from pandas import concat

template = read_csv("/Users/afreiburger/Documents/CommScores/examples/fastq_reads_noninterleaved.csv")
template.drop(2, inplace=True)
template.columns = template.iloc[0]
display(template)
new_rows_df = DataFrame(rows, columns=template.columns)
display(new_rows_df)
new_rows_df.to_csv("FASTQ_template.csv")
# total_df = concat([template, new_rows_df])
# display(total_df)

,fastq_fwd_staging_file_name,fastq_rev_staging_file_name,sequencing_tech,name,single_genome,read_orientation_outward,insert_size_std_dev,insert_size_mean
0,fastq_fwd_staging_file_name,fastq_rev_staging_file_name,sequencing_tech,name,single_genome,read_orientation_outward,insert_size_std_dev,insert_size_mean
1,Forward/left FASTQ file path,Reverse/right FASTQ file path,Sequencing technology,Reads object name,Single genome,Reads orientation outward,St. dev. of insert size,Mean insert size


,fastq_fwd_staging_file_name,fastq_rev_staging_file_name,sequencing_tech,name,single_genome,read_orientation_outward,insert_size_std_dev,insert_size_mean
0,240412_Gupta_Hanke_fastqs/PR1_S14_R1_001.fastq.gz,240412_Gupta_Hanke_fastqs/PR1_S14_R2_001.fastq.gz,Illumina,PR1_S14,1,0,NaN,NaN
1,240412_Gupta_Hanke_fastqs/EL_S16_R1_001.fastq.gz,240412_Gupta_Hanke_fastqs/EL_S16_R2_001.fastq.gz,Illumina,EL_S16,1,0,NaN,NaN
2,240412_Gupta_Hanke_fastqs/EA_S1_R1_001.fastq.gz,240412_Gupta_Hanke_fastqs/EA_S1_R2_001.fastq.gz,Illumina,EA_S1,1,0,NaN,NaN
3,240412_Gupta_Hanke_fastqs/PR2_S17_R1_001.fastq.gz,240412_Gupta_Hanke_fastqs/PR2_S17_R2_001.fastq.gz,Illumina,PR2_S17,1,0,NaN,NaN
4,240412_Gupta_Hanke_fastqs/RP2_S7_R1_001.fastq.gz,240412_Gupta_Hanke_fastqs/RP2_S7_R2_001.fastq.gz,Illumina,RP2_S7,1,0,NaN,NaN
5,240412_Gupta_Hanke_fastqs/PAr_S19_R1_001.fastq.gz,240412_Gupta_Hanke_fastqs/PAr_S19_R2_001.fastq.gz,Illumina,PAr_S19,1,0,NaN,NaN
6,240412_Gupta_Hanke_fastqs/PK_S18_R1_001.fastq.gz,240412_Gupta_Hanke_fastqs/PK_S18_R2_001.fastq.gz,Illumina,PK_S18,1,0,NaN,NaN
7,240412_Gupta_Hanke_fastqs/PAl_S11_R1_001.fastq.gz,240412_Gupta_Hanke_fastqs/PAl_S11_R2_001.fastq.gz,Illumina,PAl_S11,1,0,NaN,NaN
8,240412_Gupta_Hanke_fastqs/BI_S3_R1_001.fastq.gz,240412_Gupta_Hanke_fastqs/BI_S3_R2_001.fastq.gz,Illumina,BI_S3,1,0,NaN,NaN
9,240412_Gupta_Hanke_fastqs/PAg3_S12_R1_001.fast...,240412_Gupta_Hanke_fastqs/PAg3_S12_R2_001.fast...,Illumina,PAg3_S12,1,0,NaN,NaN


# test larger 10+ communities

In [1]:
import cobrakbase
# with open("/Users/afreiburger/Documents/kbase_token.txt") as token_file:
# with open("C:/Users/Andrew Freiburger/Documents/Argonne/kbase_token.txt") as token_file:
kbase_api = cobrakbase.KBaseAPI("QZKPWF7UPWRL4VY5RVCT2KSXYNQT3LUJ")

models_strings = ['Lysobacter_sp._CF310_2616644831.fna.RAST.glm4ec.mdl (v1)',
 'Arthrobacter_sp._OV407_2609459738.fna.RAST.gim4ec.mdl (v1)',
 'Rhizobium_sp._PDC82_2690315628.fna.RAST.glm4ec.madl (v1)',
 'Duganella_sp._CF402_2616644845.fna.RAST.glm4ec.mdl (v1)',
 'Acinetobacter_sp._YR461_2615840642.fna.RAST.glm4ec.mdl (v1)',
 'Stenotrophomonas_sp._CF319_2758568591.fna.RAST.glm4ec.mdl (v1)',
 'Sphingobium_sp._YR768_2687453792.fna.RAST.gim4ec.mdl (v1)',
 '',
 'Ensifer_sp._YR511_2615840643.fna.RAST.glm4ec.mdl (v1)',
 '',
 'Acinetobacter_sp._YR461_2615840642.fna.RAST.glm4ec.mdl (v1)',
 'Pseudoxanthomonas_sp._CF385_2690315626.fna.RAST.glm4ec.madl (v1)',
 'Elizabethkingia_sp._YR214_2738541273.fna.RAST.glm4ec.mdl (v1)',
 'Dyella_sp._OK004_2609459631.fna.RAST.glm4ec.mdl (v1)',
 'Plantibacter_sp._YR521_2758568592.fna.RAST.glm4ec.mdl (v1)',
 'Pseudomonas_sp._OV144_2757320538.fna.RAST.glm4ec.madl (v1)',
 'Sphingobium_sp._YR657_2687453679.fna.RAST.glm4ec.madl (v1)',
 'Caulobacter_sp._OV484_2585428106.fna.RAST.glm4ec.madl (v1)',
 'Rahnella_aquatilis_OV588_2585427592.fna.RAST.glm4ec.mdl (v1)',
 'Pseudomonas_sp._GM17_2511231006.fna.RAST.glm4ec.madl (v1)',
 'Agromyces_sp._OV415_2757320520.fna.RAST.glm4ec.madl (v1)',
 '',
 'Albidiferax_sp._OV413_2615840603.fna.RAST.glm4ec.mdl (v1)',
 '']
models_strings = [model.replace(" (v1)", "") for model in models_strings]

models = [kbase_api.get_from_ws(model_name, 179194) for model_name in models_strings]
display(models)

modelseedpy 0.2.2
cobrakbase 0.3.1


KeyError: 'role'

In [ ]:
%run ../commscores/report.py
table, mets = report_generation(models)

In [4]:

from json import dump

with open("mro_results.json", 'w') as jsonOut:
    dump({"Pantoea_sp._YR343_2511231025.fna.RAST.mdl---Rhizobium_sp._CF142_2510917030.fna.RAST.mdl": (0.0, 0,15),
 "Rhizobium_sp._CF142_2510917030.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl": (0.0, 0, 11),
 "Rhizobium_sp._CF142_2510917030.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl--mets": [],
 "Pantoea_sp._YR343_2511231025.fna.RAST.mdl---Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl": (66.66666666666667, 10, 15),
 "Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl": (52.63157894736842, 10, 19),
 "Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl--mets": [
  "cpd00254",
  "cpd10515",
  "cpd00034",
  "cpd00063",
  "cpd00058",
  "cpd00099",
  "cpd00030",
  "cpd00027",
  "cpd00205",
  "cpd00149"],
 "Pantoea_sp._YR343_2511231025.fna.RAST.mdl---Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl": (6.666666666666667,
  1,
  15),
 "Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl": (14.285714285714286,
  1,
  7),
 "Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl--mets": ["cpd10515"],
 "Rhizobium_sp._CF142_2510917030.fna.RAST.mdl---Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl": (9.090909090909092,
  1,
  11),
 "Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl---Rhizobium_sp._CF142_2510917030.fna.RAST.mdl": (5.2631578947368425,
  1,
  19),
 "Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl---Rhizobium_sp._CF142_2510917030.fna.RAST.mdl--mets": ["cpd00012"],
 "Rhizobium_sp._CF142_2510917030.fna.RAST.mdl---Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl": (18.181818181818183,
  2,
  11),
 "Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl---Rhizobium_sp._CF142_2510917030.fna.RAST.mdl": (28.571428571428573,
  2,
  7),
 "Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl---Rhizobium_sp._CF142_2510917030.fna.RAST.mdl--mets": ["cpd00322",
  "cpd01329"],
 "Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl---Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl": (5.2631578947368425,
  1,
  19),
 "Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl---Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl": (14.285714285714286,
  1,
  7),
 "Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl---Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl--mets": ["cpd10515"]}, jsonOut, indent=3)

display({'Pantoea_sp._YR343_2511231025.fna.RAST.mdl---Rhizobium_sp._CF142_2510917030.fna.RAST.mdl': (0.0, 0, 15), 'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl': (0.0, 0, 11), 'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl--mets': [], 'Pantoea_sp._YR343_2511231025.fna.RAST.mdl---Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl': (66.66666666666667, 10, 15), 'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl': (52.63157894736842, 10, 19), 'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl--mets': ['cpd00254', 'cpd10515', 'cpd00034', 'cpd00063', 'cpd00058', 'cpd00099', 'cpd00030', 'cpd00027', 'cpd00205', 'cpd00149'], 'Pantoea_sp._YR343_2511231025.fna.RAST.mdl---Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl': (6.666666666666667, 1, 15), 'Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl': (14.285714285714286, 1, 7), 'Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl--mets': ['cpd10515'], 'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl---Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl': (9.090909090909092, 1, 11), 'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl---Rhizobium_sp._CF142_2510917030.fna.RAST.mdl': (5.2631578947368425, 1, 19), 'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl---Rhizobium_sp._CF142_2510917030.fna.RAST.mdl--mets': ['cpd00012'], 'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl---Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl': (18.181818181818183, 2, 11), 'Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl---Rhizobium_sp._CF142_2510917030.fna.RAST.mdl': (28.571428571428573, 2, 7), 'Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl---Rhizobium_sp._CF142_2510917030.fna.RAST.mdl--mets': ['cpd00322', 'cpd01329'], 'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl---Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl': (5.2631578947368425, 1, 19), 'Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl---Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl': (14.285714285714286, 1, 7), 'Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl---Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl--mets': ['cpd10515']})

{'Pantoea_sp._YR343_2511231025.fna.RAST.mdl---Rhizobium_sp._CF142_2510917030.fna.RAST.mdl': (0.0,
  0,
  15),
 'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl': (0.0,
  0,
  11),
 'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl--mets': [],
 'Pantoea_sp._YR343_2511231025.fna.RAST.mdl---Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl': (66.66666666666667,
  10,
  15),
 'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl': (52.63157894736842,
  10,
  19),
 'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl--mets': ['cpd00254',
  'cpd10515',
  'cpd00034',
  'cpd00063',
  'cpd00058',
  'cpd00099',
  'cpd00030',
  'cpd00027',
  'cpd00205',
  'cpd00149'],
 'Pantoea_sp._YR343_2511231025.fna.RAST.mdl---Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl': (6.666666666666667,
  1,
  15),
 'Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl

In [5]:
dic = {'Pantoea_sp._YR343_2511231025.fna.RAST.mdl---Rhizobium_sp._CF142_2510917030.fna.RAST.mdl': ([], {'EX_cpd00149_e0': 0.00016000406624403266, 'EX_cpd10515_e0': 0.00048001219873209814, 'EX_cpd00058_e0': 0.00016000406624403271, 'EX_cpd00099_e0': 0.0001600040662440327, 'EX_cpd10516_e0': 0.0001600040662440423, 'EX_cpd00048_e0': 0.02080444907177458, 'EX_cpd00007_e0': 0.8312926730795334, 'EX_cpd00030_e0': 0.0001600040662440327, 'EX_cpd00254_e0': 0.00016000406624403266, 'EX_cpd00063_e0': 0.0001600040662440327, 'EX_cpd00205_e0': 0.0001600040662440327, 'EX_cpd00009_e0': 0.06609696295927116, 'EX_cpd00027_e0': 1.5309013820474202, 'EX_cpd00013_e0': 0.7992716580596883, 'EX_cpd00034_e0': 0.00016000406624403271}), 'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl': ([], {'EX_cpd00067_e0': 7.069786749454685e-16, 'EX_cpd00156_e0': 6.646010303720901e-16, 'EX_cpd00322_e0': 3.3574895661214506e-16, 'EX_cpd09878_e0': 3.536760818553649e-17, 'EX_cpd00012_e0': 1.1003962539422529e-15, 'EX_cpd01981_e0': 5.894601364255869e-18, 'EX_cpd03048_e0': 2.3578405457023e-17, 'EX_cpd01329_e0': 3.196897846544275e-16, 'EX_cpd11593_e0': 2.62693791155145e-16, 'EX_cpd01017_e0': 1.6403916850458471e-15, 'EX_cpd11590_e0': 7.452919085566754e-16}), 'Pantoea_sp._YR343_2511231025.fna.RAST.mdl---Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl': (['cpd00034', 'cpd00099', 'cpd00149', 'cpd00030', 'cpd00058', 'cpd00063', 'cpd00254', 'cpd00205', 'cpd10515', 'cpd00027'], {'EX_cpd00149_e0': 0.00016000406624403266, 'EX_cpd10515_e0': 0.00048001219873209814, 'EX_cpd00058_e0': 0.00016000406624403271, 'EX_cpd00099_e0': 0.0001600040662440327, 'EX_cpd10516_e0': 0.0001600040662440423, 'EX_cpd00048_e0': 0.02080444907177458, 'EX_cpd00007_e0': 0.8312926730795334, 'EX_cpd00030_e0': 0.0001600040662440327, 'EX_cpd00254_e0': 0.00016000406624403266, 'EX_cpd00063_e0': 0.0001600040662440327, 'EX_cpd00205_e0': 0.0001600040662440327, 'EX_cpd00009_e0': 0.06609696295927116, 'EX_cpd00027_e0': 1.5309013820474202, 'EX_cpd00013_e0': 0.7992716580596883, 'EX_cpd00034_e0': 0.00016000406624403271}), 'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl': (['cpd00034', 'cpd00099', 'cpd00149', 'cpd00030', 'cpd00058', 'cpd00063', 'cpd00254', 'cpd00205', 'cpd10515', 'cpd00027'], {'EX_cpd00149_e0': 6.925568785604016e-17, 'EX_cpd00028_e0': 6.925568785604016e-17, 'EX_cpd00012_e0': 1.2488182842261929e-13, 'EX_cpd10515_e0': 2.077670635681205e-16, 'EX_cpd00058_e0': 6.925568785604016e-17, 'EX_cpd00099_e0': 6.925568785604016e-17, 'EX_cpd00092_e0': 8.657252899185424e-13, 'EX_cpd00030_e0': 6.925568785604016e-17, 'EX_cpd00254_e0': 6.925568785604016e-17, 'EX_cpd00063_e0': 6.925568785604016e-17, 'EX_cpd00039_e0': 1.2118400475306734e-14, 'EX_cpd01080_e0': 8.588682904711969e-15, 'EX_cpd00069_e0': 5.110652431792038e-15, 'EX_cpd00268_e0': 7.53442525761958e-13, 'EX_cpd00107_e0': 1.5682278432243698e-14, 'EX_cpd00205_e0': 6.925568785604016e-17, 'EX_cpd00027_e0': 9.470125322378724e-13, 'EX_cpd00214_e0': 6.9425533052426364e-15, 'EX_cpd00034_e0': 6.925568785604017e-17}), 'Pantoea_sp._YR343_2511231025.fna.RAST.mdl---Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl': (['cpd10515'], {'EX_cpd00149_e0': 0.00016000406624403266, 'EX_cpd10515_e0': 0.00048001219873209814, 'EX_cpd00058_e0': 0.00016000406624403271, 'EX_cpd00099_e0': 0.0001600040662440327, 'EX_cpd10516_e0': 0.0001600040662440423, 'EX_cpd00048_e0': 0.02080444907177458, 'EX_cpd00007_e0': 0.8312926730795334, 'EX_cpd00030_e0': 0.0001600040662440327, 'EX_cpd00254_e0': 0.00016000406624403266, 'EX_cpd00063_e0': 0.0001600040662440327, 'EX_cpd00205_e0': 0.0001600040662440327, 'EX_cpd00009_e0': 0.06609696295927116, 'EX_cpd00027_e0': 1.5309013820474202, 'EX_cpd00013_e0': 0.7992716580596883, 'EX_cpd00034_e0': 0.00016000406624403271}), 'Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl---Pantoea_sp._YR343_2511231025.fna.RAST.mdl': (['cpd10515'], {'EX_cpd00322_e0': 2.3929245814161e-14, 'EX_cpd00047_e0': 6.626266735862558e-13, 'EX_cpd10515_e0': 2.515266845230868e-16, 'EX_cpd15604_e0': 1.3831453497511894e-13, 'EX_cpd01329_e0': 2.747233792911582e-13, 'EX_cpd00001_e0': 3.656272355096841e-12, 'EX_cpd11585_e0': 3.103358959851718e-12}), 'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl---Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl': (['cpd00012'], {'EX_cpd00067_e0': 7.069786749454685e-16, 'EX_cpd00156_e0': 6.646010303720901e-16, 'EX_cpd00322_e0': 3.3574895661214506e-16, 'EX_cpd09878_e0': 3.536760818553649e-17, 'EX_cpd00012_e0': 1.1003962539422529e-15, 'EX_cpd01981_e0': 5.894601364255869e-18, 'EX_cpd03048_e0': 2.3578405457023e-17, 'EX_cpd01329_e0': 3.196897846544275e-16, 'EX_cpd11593_e0': 2.62693791155145e-16, 'EX_cpd01017_e0': 1.6403916850458471e-15, 'EX_cpd11590_e0': 7.452919085566754e-16}), 'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl---Rhizobium_sp._CF142_2510917030.fna.RAST.mdl': (['cpd00012'], {'EX_cpd00149_e0': 6.925568785604016e-17, 'EX_cpd00028_e0': 6.925568785604016e-17, 'EX_cpd00012_e0': 1.2488182842261929e-13, 'EX_cpd10515_e0': 2.077670635681205e-16, 'EX_cpd00058_e0': 6.925568785604016e-17, 'EX_cpd00099_e0': 6.925568785604016e-17, 'EX_cpd00092_e0': 8.657252899185424e-13, 'EX_cpd00030_e0': 6.925568785604016e-17, 'EX_cpd00254_e0': 6.925568785604016e-17, 'EX_cpd00063_e0': 6.925568785604016e-17, 'EX_cpd00039_e0': 1.2118400475306734e-14, 'EX_cpd01080_e0': 8.588682904711969e-15, 'EX_cpd00069_e0': 5.110652431792038e-15, 'EX_cpd00268_e0': 7.53442525761958e-13, 'EX_cpd00107_e0': 1.5682278432243698e-14, 'EX_cpd00205_e0': 6.925568785604016e-17, 'EX_cpd00027_e0': 9.470125322378724e-13, 'EX_cpd00214_e0': 6.9425533052426364e-15, 'EX_cpd00034_e0': 6.925568785604017e-17}), 'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl---Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl': (['cpd01329', 'cpd00322'], {'EX_cpd00067_e0': 7.069786749454685e-16, 'EX_cpd00156_e0': 6.646010303720901e-16, 'EX_cpd00322_e0': 3.3574895661214506e-16, 'EX_cpd09878_e0': 3.536760818553649e-17, 'EX_cpd00012_e0': 1.1003962539422529e-15, 'EX_cpd01981_e0': 5.894601364255869e-18, 'EX_cpd03048_e0': 2.3578405457023e-17, 'EX_cpd01329_e0': 3.196897846544275e-16, 'EX_cpd11593_e0': 2.62693791155145e-16, 'EX_cpd01017_e0': 1.6403916850458471e-15, 'EX_cpd11590_e0': 7.452919085566754e-16}), 'Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl---Rhizobium_sp._CF142_2510917030.fna.RAST.mdl': (['cpd01329', 'cpd00322'], {'EX_cpd00322_e0': 2.3929245814161e-14, 'EX_cpd00047_e0': 6.626266735862558e-13, 'EX_cpd10515_e0': 2.515266845230868e-16, 'EX_cpd15604_e0': 1.3831453497511894e-13, 'EX_cpd01329_e0': 2.747233792911582e-13, 'EX_cpd00001_e0': 3.656272355096841e-12, 'EX_cpd11585_e0': 3.103358959851718e-12}), 'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl---Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl': (['cpd10515'], {'EX_cpd00149_e0': 6.925568785604016e-17, 'EX_cpd00028_e0': 6.925568785604016e-17, 'EX_cpd00012_e0': 1.2488182842261929e-13, 'EX_cpd10515_e0': 2.077670635681205e-16, 'EX_cpd00058_e0': 6.925568785604016e-17, 'EX_cpd00099_e0': 6.925568785604016e-17, 'EX_cpd00092_e0': 8.657252899185424e-13, 'EX_cpd00030_e0': 6.925568785604016e-17, 'EX_cpd00254_e0': 6.925568785604016e-17, 'EX_cpd00063_e0': 6.925568785604016e-17, 'EX_cpd00039_e0': 1.2118400475306734e-14, 'EX_cpd01080_e0': 8.588682904711969e-15, 'EX_cpd00069_e0': 5.110652431792038e-15, 'EX_cpd00268_e0': 7.53442525761958e-13, 'EX_cpd00107_e0': 1.5682278432243698e-14, 'EX_cpd00205_e0': 6.925568785604016e-17, 'EX_cpd00027_e0': 9.470125322378724e-13, 'EX_cpd00214_e0': 6.9425533052426364e-15, 'EX_cpd00034_e0': 6.925568785604017e-17}), 'Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl---Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl': (['cpd10515'], {'EX_cpd00322_e0': 2.3929245814161e-14, 'EX_cpd00047_e0': 6.626266735862558e-13, 'EX_cpd10515_e0': 2.515266845230868e-16, 'EX_cpd15604_e0': 1.3831453497511894e-13, 'EX_cpd01329_e0': 2.747233792911582e-13, 'EX_cpd00001_e0': 3.656272355096841e-12, 'EX_cpd11585_e0': 3.103358959851718e-12})}


from json import dump

with open("mro_results2.json", 'w') as jsonOut:
    dump(dic, jsonOut, indent=3)

# test CommScores

In [1]:
# from cobra.io import read_sbml_model, write_sbml_model
# from glob import glob
# from json import load
# from os import path

# test_dir = "../test/"

# models = []
# for model_path in glob(f"{test_dir}*.xml"):
#     model = read_sbml_model(model_path)
#     model.id = path.basename(model_path).replace(".xml", '')
#     print(model.id)
#     models.append(model)
#     print(model.slim_optimize())
#     # write_sbml_model(model, test_dir+model.id+".xml")
# with open(f"{test_dir}carbonDGlucose.json", "r") as jsonIn:
#     media = load(jsonIn)

'' is not a valid SBML 'SId'.


Pantoea_sp._YR343_2511231025.fna.RAST.mdl


'' is not a valid SBML 'SId'.


93.06480403074329
Rhizobium_sp._CF142_2510917030.fna.RAST.mdl


'' is not a valid SBML 'SId'.


58.346758104559726
Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl
82.61277856295986


'' is not a valid SBML 'SId'.


Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl
70.62333478562601


In [8]:
print(dir(models[0]))
print([cons.name for cons in models[0].constraints])

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_annotation', '_compartments', '_contexts', '_id', '_populate_solver', '_repr_html_', '_sbml', '_set_id_with_model', '_solver', '_tolerance', 'add_boundary', 'add_cons_vars', 'add_groups', 'add_metabolites', 'add_reactions', 'annotation', 'boundary', 'compartments', 'constraints', 'copy', 'demands', 'exchanges', 'genes', 'get_associated_groups', 'groups', 'id', 'medium', 'merge', 'metabolites', 'name', 'notes', 'objective', 'objective_direction', 'optimize', 'problem', 'reactions', 'remove_cons_vars', 'remove_groups', 'remove_metabolites', 'remove_reactions', 'repair', 'sinks', 'slim_optimi

In [5]:
sol = models[0].optimize()
print(type(dict(sol.fluxes.items())), dir(sol.fluxes))

<class 'dict'> ['T', '_AXIS_LEN', '_AXIS_ORDERS', '_AXIS_TO_AXIS_NUMBER', '_HANDLED_TYPES', '__abs__', '__add__', '__and__', '__annotations__', '__array__', '__array_priority__', '__array_ufunc__', '__bool__', '__class__', '__column_consortium_standard__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__divmod__', '__doc__', '__eq__', '__finalize__', '__float__', '__floordiv__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__iand__', '__ifloordiv__', '__imod__', '__imul__', '__init__', '__init_subclass__', '__int__', '__invert__', '__ior__', '__ipow__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__lt__', '__matmul__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pandas_priority__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__

In [8]:
# %run /Users/afreiburger/Documents/ModelSEEDpy/modelseedpy/core/msminimalmedia.py
# %run /Users/afreiburger/Documents/CommScores/commscores/commscoresutil.py
# from commscores import CommScoresUtil, mro

# for model in models:
#     MSMinimalMedia.minimize_flux(model, 0.1)

# CommScoresUtil._get_media(model_s_=models)




# %run /Users/andrewfreiburger/Documents/Research/CommScores/commscores/scores/mro.py
# display(mro(models, climit=True, o2limit=True))
# %run /Users/andrewfreiburger/Documents/Research/CommScores/commscores/scores/mip.py
# display(mip(models, environment=media, climit=True, o2limit=True))


# %run /Users/andrewfreiburger/Documents/Research/CommScores/commscores/scores/bss.py
# result = bss(models, environment=media, climit=True, o2limit=True)

# %run /Users/andrewfreiburger/Documents/Research/CommScores/commscores/scores/gyd.py
# result = gyd(models, environment=media, climit=True, o2limit=True)

%run /Users/andrewfreiburger/Documents/Research/CommScores/commscores/scores/pc.py
result = pc(models, environment=media, climit=True, o2limit=True)

# print(result.keys())
display(result)

bio1 from Pantoea_sp._YR343_2511231025.fna.RAST.mdl becomes bio2
bio1 from Rhizobium_sp._CF142_2510917030.fna.RAST.mdl becomes bio3
bio1 from Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl becomes bio4
bio1 from Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl becomes bio5
Community objective 1.0*bio1 - 1.0*bio1_reverse_b18f7
Pantoea_sp._YR343_2511231025.fna.RAST.mdl biomass compound: cpd11416_c1
Rhizobium_sp._CF142_2510917030.fna.RAST.mdl biomass compound: cpd11416_c2
Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl biomass compound: cpd11416_c3
Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl biomass compound: cpd11416_c4
Pantoea_sp._YR343_2511231025.fna.RAST.mdl 300 0.25
Rhizobium_sp._CF142_2510917030.fna.RAST.mdl 300 0.25
Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl 300 0.25
Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl 300 0.25


(0.35592332391746084,
 {'Pantoea_sp._YR343_2511231025.fna.RAST.mdl': 0.35093891232328933,
  'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl': 0.36326061235963547,
  'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl': 0.35395522175405186,
  'Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl': 0.3557685553911495},
 {'Pantoea_sp._YR343_2511231025.fna.RAST.mdl': 0.4222354656743602,
  'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl': 0.42223546567436016,
  'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl': 0.42223546567436016,
  'Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl': 0.4222354656743601},
 'competitive')

In [11]:
media

{'EX_cpd03424_e0': 1000,
 'EX_cpd00215_e0': 1000,
 'EX_cpd00028_e0': 1000,
 'EX_cpd10515_e0': 1000,
 'EX_cpd00030_e0': 1000,
 'EX_cpd00149_e0': 1000,
 'EX_cpd00058_e0': 1000,
 'EX_cpd00099_e0': 1000,
 'EX_cpd00007_e0': 1000,
 'EX_cpd00034_e0': 1000,
 'EX_cpd00156_e0': 1000,
 'EX_cpd00249_e0': 1000,
 'EX_cpd00092_e0': 1000,
 'EX_cpd00069_e0': 1000,
 'EX_cpd00065_e0': 1000,
 'EX_cpd00184_e0': 1000,
 'EX_cpd00161_e0': 1000,
 'EX_cpd00048_e0': 1000,
 'EX_cpd00054_e0': 1000,
 'EX_cpd00220_e0': 1000,
 'EX_cpd00129_e0': 1000,
 'EX_cpd00644_e0': 1000,
 'EX_cpd00009_e0': 1000,
 'EX_cpd00066_e0': 1000,
 'EX_cpd00218_e0': 1000,
 'EX_cpd00971_e0': 1000,
 'EX_cpd00254_e0': 1000,
 'EX_cpd00060_e0': 1000,
 'EX_cpd00039_e0': 1000,
 'EX_cpd00107_e0': 1000,
 'EX_cpd00205_e0': 1000,
 'EX_cpd00246_e0': 1000,
 'EX_cpd00322_e0': 1000,
 'EX_cpd00226_e0': 1000,
 'EX_cpd00119_e0': 1000,
 'EX_cpd00531_e0': 1000,
 'EX_cpd00001_e0': 1000,
 'EX_cpd00067_e0': 1000,
 'EX_cpd00033_e0': 1000,
 'EX_cpd00023_e0': 1000,


In [ ]:

%run /Users/andrewfreiburger/Documents/Research/CommScores/commscores/report.py
result = report_generation(models, environments={"R2A": media}, climit=120, o2limit=40, kinCoef=750)

# %run /Users/andrewfreiburger/Documents/Research/CommScores/commscores/calculate_scores.py
# result = calculate_scores(models, environments={"R2A": media}, climit=120, o2limit=40)

The minimal flux media for Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl consists of 35 compounds and a 1.7749988124291822 total influx, with a growth value of 0.09999999999976701
The minimal flux media for Pantoea_sp._YR343_2511231025.fna.RAST.mdl consists of 30 compounds and a 1.579982576026846 total influx, with a growth value of 0.10000000000022322


In [3]:
display(result[0])
result[0].to_csv("kinCoef_750.csv")

,model1,model2,media,monoculture growth model1,monoculture growth model2,coculture growth model1,coculture growth model2,community growth,MRO_model1,MRO_model2,...,MIP_model1 (costless),MIP_model2 (costless),BSS_model1,BSS_model2,PC_model1,PC_model2,PC_comm,BIT,GYD1,GYD2
0,Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl,Pantoea_sp._YR343_2511231025.fna.RAST.mdl,R2A,1.1929,1.2032,0.15302,2.2498,2.4028,51.429% (18/35),60.000% (18/30),...,2 (0),2 (0),50.0%,60.0%,0.12828,1.8698,1.0028,parasitism,-0.008634,0.008561
1,Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl,Streptomyces_mirabilis_YR139_2582581314.fna.RA...,R2A,1.1929,1.1868,0.23202,2.1569,2.3889,45.714% (16/35),64.000% (16/25),...,2 (1),1 (0),48.0%,65.714%,0.1945,1.8174,1.0039,parasitism,0.005114,-0.00514
2,Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl,Rhizobium_sp._CF142_2510917030.fna.RAST.mdl,R2A,1.1929,1.1623,0.70182,1.694,2.3958,45.714% (16/35),69.565% (16/23),...,4 (0),2 (0),47.826%,60.0%,0.58833,1.4574,1.0172,parasitism,0.025652,-0.026327
3,Pantoea_sp._YR343_2511231025.fna.RAST.mdl,Streptomyces_mirabilis_YR139_2582581314.fna.RA...,R2A,1.2032,1.1868,0.068279,2.3231,2.3914,53.333% (16/30),64.000% (16/25),...,0 (0),1 (0),48.0%,53.333%,0.056748,1.9575,1.0006,parasitism,0.01363,-0.013819
4,Pantoea_sp._YR343_2511231025.fna.RAST.mdl,Rhizobium_sp._CF142_2510917030.fna.RAST.mdl,R2A,1.2032,1.1623,2.2713,0.12962,2.4009,50.000% (15/30),65.217% (15/23),...,1 (0),0 (0),52.174%,53.333%,1.8877,0.11152,1.015,parasitism,0.033993,-0.035189
5,Streptomyces_mirabilis_YR139_2582581314.fna.RA...,Rhizobium_sp._CF142_2510917030.fna.RAST.mdl,R2A,1.1868,1.1623,1.8101,0.57332,2.3834,76.000% (19/25),82.609% (19/23),...,1 (0),1 (0),52.174%,40.0%,1.5252,0.49326,1.0146,parasitism,0.020644,-0.021079


In [4]:
%run /Users/afreiburger/Documents/CommScores/test/test_commscores.py

test_mro()

Exception: File `'/Users/afreiburger/Documents/CommScores/test/test_commscores.py'` not found.

In [3]:
from modelseedpy import MSCommunity
comm = MSCommunity(member_models=models)
print(dir(comm))

bio1 from Pantoea_sp._YR343_2511231025.fna.RAST.mdl becomes bio2
bio1 from Rhizobium_sp._CF142_2510917030.fna.RAST.mdl becomes bio3
bio1 from Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl becomes bio4
bio1 from Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl becomes bio5
Community objective 1.0*bio1 - 1.0*bio1_reverse_b18f7
Pantoea_sp._YR343_2511231025.fna.RAST.mdl biomass compound: cpd11416_c1
Rhizobium_sp._CF142_2510917030.fna.RAST.mdl biomass compound: cpd11416_c2
Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl biomass compound: cpd11416_c3
Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl biomass compound: cpd11416_c4
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_com

In [2]:
%run ../commscores/scores/mip.py
mip_results = mip(models, environment=media, costless=True)

from json import dump
with open("../test/mip_results.json", "w") as jsonOut:
    dump(mip_results, jsonOut, indent=3)

modelseedpy 0.2.2


/Users/afreiburger/Documents/CommScores/commscores/scores/mip.py:48: SyntaxWarning: invalid escape sequence '\['
  cross_fed_exIDs = [re.sub("(root\['|'\])", "", x) for x in interact_diff["dictionary_item_removed"]]
/Users/afreiburger/Documents/CommScores/commscores/utils.py:63: SyntaxWarning: invalid escape sequence '\s'
  rm_costless = re.compile("(\s\(.+\))")


bio1 from Pantoea_sp._YR343_2511231025.fna.RAST.mdl becomes bio2
bio1 from Rhizobium_sp._CF142_2510917030.fna.RAST.mdl becomes bio3
bio1 from Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl becomes bio4
bio1 from Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl becomes bio5
Community objective 1.0*bio1 - 1.0*bio1_reverse_b18f7
MIP_comm 1.0*bio1 - 1.0*bio1_reverse_b18f7
Non-interacting community, minimize transporters	The minimal flux media for MIP_comm consists of 63 compounds and a 3889.2696475905836 total influx, with a growth value of 0.0999999999987396
Interacting community, minimize exchanges	The minimal flux media for MIP_comm consists of 30 compounds and a 0.9606586797672002 total influx, with a growth value of 0.099999999999799


In [7]:
%run ../commscores/scores/gyd.py

gyd_results = gyd(models)

from json import dump
with open("../test/gyd_results.json", "w") as jsonOut:
    dump(gyd_results, jsonOut, indent=3)

Pantoea_sp._YR343_2511231025.fna.RAST.mdl ++ Rhizobium_sp._CF142_2510917030.fna.RAST.mdl 93.06480403074247 58.34675810455966
Pantoea_sp._YR343_2511231025.fna.RAST.mdl ++ Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl 93.06480403074247 82.61277856295995
Pantoea_sp._YR343_2511231025.fna.RAST.mdl ++ Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl 93.06480403074247 70.62333478562508
Rhizobium_sp._CF142_2510917030.fna.RAST.mdl ++ Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl 58.34675810455966 82.61277856295995
Rhizobium_sp._CF142_2510917030.fna.RAST.mdl ++ Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl 58.34675810455966 70.62333478562508
Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl ++ Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl 82.61277856295995 70.62333478562508


In [9]:
gyd_results

{'Pantoea_sp._YR343_2511231025.fna.RAST.mdl ++ Rhizobium_sp._CF142_2510917030.fna.RAST.mdl': (0.3730523723524336,
  0.5950295621218015,
  93.06480403074247,
  58.34675810455966),
 'Pantoea_sp._YR343_2511231025.fna.RAST.mdl ++ Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl': (0.1123091116629855,
  0.12651826569199504,
  93.06480403074247,
  82.61277856295995),
 'Pantoea_sp._YR343_2511231025.fna.RAST.mdl ++ Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl': (0.2411380916646449,
  0.3177628090380859,
  93.06480403074247,
  70.62333478562508),
 'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl ++ Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl': (0.41589320892370113,
  0.29373204582275286,
  58.34675810455966,
  82.61277856295995),
 'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl ++ Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl': (0.21040717736305609,
  0.173831733071379,
  58.34675810455966,
  70.62333478562508),
 'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl ++ Streptomyces_mirabilis_YR139

In [9]:
%run ../commscores/scores/pc.py

pc_results = pc(models)

display(pc_results)

from json import dump
with open("../test/pc_results.json", "w") as jsonOut:
    dump(pc_results, jsonOut, indent=3)

bio1 from Pantoea_sp._YR343_2511231025.fna.RAST.mdl becomes bio2
bio1 from Rhizobium_sp._CF142_2510917030.fna.RAST.mdl becomes bio3
bio1 from Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl becomes bio4
bio1 from Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl becomes bio5
Community objective 1.0*bio1 - 1.0*bio1_reverse_b18f7
Pantoea_sp._YR343_2511231025.fna.RAST.mdl biomass compound: cpd11416_c1
Rhizobium_sp._CF142_2510917030.fna.RAST.mdl biomass compound: cpd11416_c2
Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl biomass compound: cpd11416_c3
Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl biomass compound: cpd11416_c4


(0.7729382747623191,
 {'Pantoea_sp._YR343_2511231025.fna.RAST.mdl': 0.9899005080423035,
  'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl': 0.7107731787472508,
  'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl': 0.8421961070629606,
  'Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl': 0.4573768822878911},
 {'Pantoea_sp._YR343_2511231025.fna.RAST.mdl': 92.12489679088938,
  'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl': 41.47131072757479,
  'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl': 69.57616049937927,
  'Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl': 32.301480681023165},
 'amensalism')

In [2]:
%run ../commscores/scores/cip.py

cip_results = cip(member_models=models, environment=media)

display(cip_results)

from json import dump
with open("../test/cip_results.json", "w") as jsonOut:
    dump(cip_results, jsonOut, indent=3)

modelseedpy 0.2.2


(['cpd01981', 'cpd00001', 'cpd00067', 'cpd00011'], 4)

In [20]:
%run ../commscores/scores/bss.py

bss_results = bss(models, environment=media)

display(bss_results)

from json import dump
with open("../test/bss_results.json", "w") as jsonOut:
    dump(bss_results, jsonOut, indent=3)

The minimal flux media for Pantoea_sp._YR343_2511231025.fna.RAST.mdl consists of 15 compounds and a 3.250287174012837 total influx, with a growth value of 0.10000000000104767
The minimal flux media for Pantoea_sp._YR343_2511231025.fna.RAST.mdl consists of 15 compounds and a 3.250287174012837 total influx, with a growth value of 0.10000000000104767
The minimal flux media for Pantoea_sp._YR343_2511231025.fna.RAST.mdl consists of 15 compounds and a 3.250287174012837 total influx, with a growth value of 0.10000000000104767
The minimal flux media for Rhizobium_sp._CF142_2510917030.fna.RAST.mdl consists of 0 compounds and a 0 total influx, with a growth value of 0.0
The minimal flux media for Rhizobium_sp._CF142_2510917030.fna.RAST.mdl consists of 0 compounds and a 0 total influx, with a growth value of 0.0
The minimal flux media for Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl consists of 0 compounds and a 0 total influx, with a growth value of -3.393752970839826e-14


{'Pantoea_sp._YR343_2511231025.fna.RAST.mdl supporting Rhizobium_sp._CF142_2510917030.fna.RAST.mdl in media0': (['cpd09027',
   'cpd03443',
   'cpd15339',
   'cpd01758',
   'cpd00348',
   'cpd00438',
   'cpd03049',
   'cpd11489',
   'cpd02663',
   'cpd11514',
   'cpd26481',
   'cpd15529',
   'cpd03470',
   'cpd02961',
   'cpd00068',
   'cpd21481',
   'cpd02979',
   'cpd15538',
   'cpd08316',
   'cpd00184',
   'cpd15418',
   'cpd02826',
   'cpd01741',
   'cpd32452',
   'cpd00822',
   'cpd00014',
   'cpd15302',
   'cpd15420',
   'cpd00989',
   'cpd15671',
   'cpd00247',
   'cpd15667',
   'cpd00524',
   'cpd00954',
   'cpd00297',
   'cpd00221',
   'cpd00781',
   'cpd15500',
   'cpd19019',
   'cpd15485',
   'cpd02498',
   'cpd15719',
   'cpd00002',
   'cpd08353',
   'cpd02124',
   'cpd00693',
   'cpd00121',
   'cpd15674',
   'cpd00050',
   'cpd15556',
   'cpd00689',
   'cpd01211',
   'cpd00235',
   'cpd03117',
   'cpd28205',
   'cpd00293',
   'cpd00045',
   'cpd15720',
   'cpd00585',
   'c

In [40]:
from cobrakbase.io.read_kbase_zip import _read_kbase_zip
from cobrakbase.core.kbase_object_factory import KBaseObjectFactory
from glob import glob

genomes = []
for genomeZip in glob("../test/*.zip"):
    print(genomeZip)
    info, data = _read_kbase_zip(genomeZip)
    completeObj = KBaseObjectFactory().create({'data': [{'info': info['info'],'provenance': [],'data': data}]}, None)
    genomes.append(completeObj)

../test/Rhizobium_sp._CF142_2510917030.fna.RAST.json (1).zip
../test/Pseudomonas_sp._GM17_2511231006.fna.RAST.json (1).zip
../test/Pantoea_sp._YR343_2511231025.fna.RAST.json (1).zip
../test/Streptomyces_mirabilis_YR139_2582581314.fna.RAST.json (1).zip


In [9]:
%run ../commscores/commscoresutil.py
# %run ../commscores/scores/fs.py

# load genomes
from glob import glob
from json import load
from os import path
genomes = {}
for genetics in glob("../test/*.fna.RAST.json"):
    baseName = path.basename(genetics).replace(".json", '')
    genomes[baseName] = load(open(genetics, 'r'))

# compute the FS score
from commscores import fs
fs_results = fs(models, annotated_genomes=genomes)
display(fs_results)

# export the FS results
from json import dump
with open("../test/fs_results.json", "w") as jsonOut:
    dump(fs_results, jsonOut, indent=3)

/Users/afreiburger/Documents/CommScores/commscores/commscoresutil.py:112: SyntaxWarning: invalid escape sequence '\s'
  rm_costless = re.compile("(\s\(.+\))")


NoSectionError: No section: 'DevEnv'

NoSectionError: No section: 'DevEnv'

In [3]:
print(media)

{'EX_cpd00149_e0': 100, 'EX_cpd00099_e0': 100, 'EX_cpd00067_e0': 100, 'EX_cpd00063_e0': 100, 'EX_cpd00058_e0': 100, 'EX_cpd00048_e0': 100, 'EX_cpd00034_e0': 100, 'EX_cpd00030_e0': 100, 'EX_cpd00013_e0': 100, 'EX_cpd00009_e0': 100, 'EX_cpd00001_e0': 100, 'EX_cpd00007_e0': 10, 'EX_cpd00205_e0': 100, 'EX_cpd00254_e0': 100, 'EX_cpd00971_e0': 100, 'EX_cpd10515_e0': 100, 'EX_cpd10516_e0': 100, 'EX_cpd11574_e0': 100, 'EX_cpd00244_e0': 100, 'EX_cpd00027_e0': 5}


In [2]:
# define the environment path 
import os
local_cobrakbase_path = os.path.join("C:", "Users", 'Andrew Freiburger',"Documents","Argonne","cobrakbase")
os.environ["HOME"] = local_cobrakbase_path

# import the KBase
import cobrakbase
with open("/Users/afreiburger/Documents/kbase_token.txt") as token_file:
# with open("C:/Users/Andrew Freiburger/Documents/Argonne/kbase_token.txt") as token_file:
    kbase_api = cobrakbase.KBaseAPI(token_file.readline())

models = [kbase_api.get_from_ws(mdl, 175377) for mdl in ["Rhizobium_sp._CF142_2510917030.fna.RAST.mdl", "Pantoea_sp._YR343_2511231025.fna.RAST.mdl",
                                                         "Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl", "Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl"]]

annotated_genomes = {
            model.id: kbase_api.get_from_ws(model.genome_ref)
            for model in models
            if hasattr(model, "genome_ref")
        }

display(annotated_genomes)

{'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl': <cobrakbase.core.kbasegenome.genome.KBaseGenome at 0x1b88fda30>,
 'Pantoea_sp._YR343_2511231025.fna.RAST.mdl': <cobrakbase.core.kbasegenome.genome.KBaseGenome at 0x4dafff7a0>,
 'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl': <cobrakbase.core.kbasegenome.genome.KBaseGenome at 0x1380f6a80>,
 'Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl': <cobrakbase.core.kbasegenome.genome.KBaseGenome at 0x4daffe8d0>}

In [11]:
print(annotated_genomes['Rhizobium_sp._CF142_2510917030.fna.RAST.mdl'].data.keys())
print(dir(annotated_genomes['Rhizobium_sp._CF142_2510917030.fna.RAST.mdl']))

dict_keys(['exclude_dict', 'data_keys', 'info', 'data', 'provenance', 'path', 'creator', 'orig_wsid', 'created', 'epoch', 'refs', 'copied', 'copy_source_inaccessible', 'features', 'id', 'scientific_name', 'domain', 'genome_tiers', 'feature_counts', 'genetic_code', 'dna_size', 'num_contigs', 'molecule_type', 'source', 'md5', 'gc_content', 'cdss', 'taxonomy', 'assembly_ref', 'taxon_ref'])
['OBJECT_TYPE', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_from_json', '_repr_html_', '_to_json', '_to_object', 'add_cdss', 'add_features', 'alias_hash', 'assembly_ref', 'cdss', 'copied', 'copy_source_inaccessible', 'created', 'creator', 'data', 'data_keys', 'dna_size', 'domain', 'epoch', '

In [5]:
print(annotated_genomes['Rhizobium_sp._CF142_2510917030.fna.RAST.mdl'].features)

[<cobrakbase.core.kbasegenome.genome_cds_feature.KBaseGenomeFeature object at 0x52deb61b0>, <cobrakbase.core.kbasegenome.genome_cds_feature.KBaseGenomeFeature object at 0x4df8a96a0>, <cobrakbase.core.kbasegenome.genome_cds_feature.KBaseGenomeFeature object at 0x4df8a8890>, <cobrakbase.core.kbasegenome.genome_cds_feature.KBaseGenomeFeature object at 0x4df8aa0f0>, <cobrakbase.core.kbasegenome.genome_cds_feature.KBaseGenomeFeature object at 0x4df8a96d0>, <cobrakbase.core.kbasegenome.genome_cds_feature.KBaseGenomeFeature object at 0x4df8a9dc0>, <cobrakbase.core.kbasegenome.genome_cds_feature.KBaseGenomeFeature object at 0x4df8a99d0>, <cobrakbase.core.kbasegenome.genome_cds_feature.KBaseGenomeFeature object at 0x4df8a9c70>, <cobrakbase.core.kbasegenome.genome_cds_feature.KBaseGenomeFeature object at 0x4df8a9790>, <cobrakbase.core.kbasegenome.genome_cds_feature.KBaseGenomeFeature object at 0x4df8a9580>, <cobrakbase.core.kbasegenome.genome_cds_feature.KBaseGenomeFeature object at 0x4df8a99a0>

In [21]:
%run ../../ModelSEEDpy/modelseedpy/community/commhelper.py


commModel = build_from_species_models(models, "comm", "CommScores_model")

/Users/afreiburger/Documents/ModelSEEDpy/modelseedpy/community/commhelper.py:26: SyntaxWarning: invalid escape sequence '\_'
  return re.sub("(\_\w\d)", "", ID)
/Users/afreiburger/Documents/ModelSEEDpy/modelseedpy/community/commhelper.py:87: SyntaxWarning: invalid escape sequence '\d'
  if re.search('^(bio)(\d+)$', rxn.id) or "biomass" in rxn.id:


bio1 from Pantoea_sp._YR343_2511231025.fna.RAST.mdl becomes bio2
bio1 from Rhizobium_sp._CF142_2510917030.fna.RAST.mdl becomes bio3
bio1 from Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl becomes bio4
bio1 from Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl becomes bio5
here2
here
1.0*bio1 - 1.0*bio1_reverse_b18f7


In [19]:
%run ../../ModelSEEDpy/modelseedpy/core/msmodelutl.py

util = MSModelUtil(models[0])
print(util.biomass_objective)

bio1


/Users/afreiburger/Documents/ModelSEEDpy/modelseedpy/core/msmodelutl.py:33: SyntaxWarning: invalid escape sequence '\d'
  if re.search("^(cpd\d+)", metabolite.id):
/Users/afreiburger/Documents/ModelSEEDpy/modelseedpy/core/msmodelutl.py:34: SyntaxWarning: invalid escape sequence '\d'
  m = re.search("^(cpd\d+)", metabolite.id)
/Users/afreiburger/Documents/ModelSEEDpy/modelseedpy/core/msmodelutl.py:39: SyntaxWarning: invalid escape sequence '\d'
  if re.search("^(cpd\d+)", item):
/Users/afreiburger/Documents/ModelSEEDpy/modelseedpy/core/msmodelutl.py:40: SyntaxWarning: invalid escape sequence '\d'
  m = re.search("^(cpd\d+)", item)
/Users/afreiburger/Documents/ModelSEEDpy/modelseedpy/core/msmodelutl.py:42: SyntaxWarning: invalid escape sequence '\d'
  elif re.search("^(cpd\d+)", metabolite.annotation[anno]):
/Users/afreiburger/Documents/ModelSEEDpy/modelseedpy/core/msmodelutl.py:43: SyntaxWarning: invalid escape sequence '\d'
  m = re.search("^(cpd\d+)", metabolite.annotation[anno])
/Use

In [16]:
print(dir(models[0]))
print(models[0].objective.expression)
print(list(models[0].objective.variables)[0].name)


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_annotation', '_compartments', '_contexts', '_id', '_populate_solver', '_repr_html_', '_sbml', '_set_id_with_model', '_solver', '_tolerance', 'add_boundary', 'add_cons_vars', 'add_groups', 'add_metabolites', 'add_reactions', 'annotation', 'boundary', 'compartments', 'constraints', 'copy', 'demands', 'exchanges', 'genes', 'get_associated_groups', 'groups', 'id', 'medium', 'merge', 'metabolites', 'name', 'notes', 'objective', 'objective_direction', 'optimize', 'problem', 'reactions', 'remove_cons_vars', 'remove_groups', 'remove_metabolites', 'remove_reactions', 'repair', 'sinks', 'slim_optimi

In [5]:
mip_output

{'../test/Pantoea_sp._YR343_2511231025.fna.RAST.mdl': ['cpd00100', 'cpd00129'],
 '../test/Rhizobium_sp._CF142_2510917030.fna.RAST.mdl': ['cpd00324'],
 '../test/Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl': ['cpd00239',
  'cpd00013'],
 '../test/Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl': ['cpd00324',
  'cpd00363',
  'cpd00129',
  'cpd00013']}

In [2]:
## test XLS loading

from pandas import read_csv

media = read_csv("~/Downloads/M9_0.1X_D-Glucose.xls", encoding='latin1')

In [3]:
media

,ÐÏà¡±á
ÌA,NaN
NaN,NaN
NaN,NaN
NaN,NaN
ÌA,NaN
...,...
NaN,NaN
NaN,NaN
NaN,NaN
NaN,NaN


In [ ]:
%run ../commscores/commscoresutil.py
%run ../commscores/report.py

# load genomes
from itertools import combinations
from glob import glob
from json import load
from os import path

genomes = {}
for genetics in glob("../test/*.fna.RAST.json"):
    baseName = path.basename(genetics).replace(".json", '.mdl')
    genomes[baseName] = load(open(genetics, 'r'))
print(list(genomes.keys()))

# all of the scores
df, mets = report_generation(models, environments=media, annotated_genomes=genomes)
display(df)

# export the total results
df.to_csv("../test/CommScores_DF.csv")
from json import dump
with open("../test/scores_output.json", "w") as jsonOut:
    dump(mets, jsonOut, indent=3)

AttributeError: 'NoneType' object has no attribute 'copy'

In [22]:
from json import load

fs_dic = load(open("../test/fs_results.json", 'r'))
display(list(fs_dic.keys()))

bss_dic = load(open("../test/bss_results.json", 'r'))
vals = list(bss_dic.values())
for i, x in enumerate(vals):
    lst, score = x
    print(list(bss_dic.keys())[i], i, len(x[0]))
    print([set(lst) == set(vals[index][0]) for index in range(len(vals)) if i != index])
    # display([Counter(x) ])

['Pantoea_sp._YR343_2511231025.fna.RAST.mdl ++ Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl',
 'Pantoea_sp._YR343_2511231025.fna.RAST.mdl ++ Rhizobium_sp._CF142_2510917030.fna.RAST.mdl',
 'Pantoea_sp._YR343_2511231025.fna.RAST.mdl ++ Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl',
 'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl ++ Rhizobium_sp._CF142_2510917030.fna.RAST.mdl',
 'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl ++ Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl',
 'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl ++ Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl']

Pantoea_sp._YR343_2511231025.fna.RAST.mdl supporting Rhizobium_sp._CF142_2510917030.fna.RAST.mdl in media0 0 907
[False, True, False, True, False]
Rhizobium_sp._CF142_2510917030.fna.RAST.mdl supporting Pantoea_sp._YR343_2511231025.fna.RAST.mdl in media0 1 865
[False, False, False, False, False]
Pantoea_sp._YR343_2511231025.fna.RAST.mdl supporting Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl in media0 2 907
[True, False, False, True, False]
Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl supporting Pantoea_sp._YR343_2511231025.fna.RAST.mdl in media0 3 890
[False, False, False, False, False]
Pantoea_sp._YR343_2511231025.fna.RAST.mdl supporting Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl in media0 4 907
[True, False, True, False, False]
Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl supporting Pantoea_sp._YR343_2511231025.fna.RAST.mdl in media0 5 858
[False, False, False, False, False]


In [38]:
%run ../commscores/scores/bss.py

output = bss(models, environment=media)
print([x[1] for x in output])

from json import dump
dump(output, open("../test/bss_results2.json", 'w'), indent=3)

The minimal flux media for Pantoea_sp._YR343_2511231025.fna.RAST.mdl consists of 7 compounds and a 5.528441881468454e-13 total influx, with a growth value of 8.329773126419129e-15
The minimal flux media for Rhizobium_sp._CF142_2510917030.fna.RAST.mdl consists of 11 compounds and a 5.840632700290867e-15 total influx, with a growth value of 7.659271649448432e-17
{'Pantoea_sp._YR343_2511231025.fna.RAST.mdl': {'media': {'EX_cpd00067_e0': 3.9404282647003083e-13, 'EX_cpd00307_e0': 1.9721488985674257e-29, 'EX_cpd08023_e0': 1.00593597978899e-14, 'EX_cpd03847_e0': 5.56115364133574e-14, 'EX_cpd00128_e0': 2.973281877434766e-14, 'EX_cpd01329_e0': 3.271187946292587e-14, 'EX_cpd15603_e0': 3.068576722829376e-14}, 'solution': <Solution 0.000 at 0x1c2fd45c0>}, 'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl': {'media': {'EX_cpd00067_e0': 7.069786749454685e-16, 'EX_cpd00156_e0': 6.646010303720901e-16, 'EX_cpd00322_e0': 3.3574895661214506e-16, 'EX_cpd09878_e0': 3.536760818553649e-17, 'EX_cpd00012_e0': 1.100

In [37]:
%run ../commscores/scores/bss.py
from json import load

bss_results = load(open("../test/bss_results.json", 'r'))
display([(k, x[1]) for k,x in bss_results.items()])

[('Pantoea_sp._YR343_2511231025.fna.RAST.mdl supporting Rhizobium_sp._CF142_2510917030.fna.RAST.mdl in media0',
  0.4666666666666667),
 ('Rhizobium_sp._CF142_2510917030.fna.RAST.mdl supporting Pantoea_sp._YR343_2511231025.fna.RAST.mdl in media0',
  0.4666666666666667),
 ('Pantoea_sp._YR343_2511231025.fna.RAST.mdl supporting Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl in media0',
  0.4666666666666667),
 ('Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl supporting Pantoea_sp._YR343_2511231025.fna.RAST.mdl in media0',
  0.4666666666666667),
 ('Pantoea_sp._YR343_2511231025.fna.RAST.mdl supporting Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl in media0',
  0.4666666666666667),
 ('Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl supporting Pantoea_sp._YR343_2511231025.fna.RAST.mdl in media0',
  0.4666666666666667)]

In [32]:

with open

[(k, x[1]) for k, x in output.items()]

[('Pantoea_sp._YR343_2511231025.fna.RAST.mdl supporting Rhizobium_sp._CF142_2510917030.fna.RAST.mdl in media0',
  0.45454545454545453),
 ('Rhizobium_sp._CF142_2510917030.fna.RAST.mdl supporting Pantoea_sp._YR343_2511231025.fna.RAST.mdl in media0',
  0.42857142857142855),
 ('Pantoea_sp._YR343_2511231025.fna.RAST.mdl supporting Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl in media0',
  0.875),
 ('Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl supporting Pantoea_sp._YR343_2511231025.fna.RAST.mdl in media0',
  0.7142857142857143),
 ('Pantoea_sp._YR343_2511231025.fna.RAST.mdl supporting Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl in media0',
  0.5714285714285714),
 ('Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl supporting Pantoea_sp._YR343_2511231025.fna.RAST.mdl in media0',
  0.7142857142857143),
 ('Rhizobium_sp._CF142_2510917030.fna.RAST.mdl supporting Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl in media0',
  0.875),
 ('Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl supporti

In [39]:
%run ../test/test_commscores.py

test_bss()

ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.


93.06480403074329
['Pantoea_sp._YR343_2511231025.fna.RAST.mdl']


ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.


58.346758104559726
['Pantoea_sp._YR343_2511231025.fna.RAST.mdl', 'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl']


ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.


82.61277856295986
['Pantoea_sp._YR343_2511231025.fna.RAST.mdl', 'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl', 'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl']


ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.


70.62333478562601
['Pantoea_sp._YR343_2511231025.fna.RAST.mdl', 'Rhizobium_sp._CF142_2510917030.fna.RAST.mdl', 'Pseudomonas_sp._GM17_2511231006.fna.RAST.mdl', 'Streptomyces_mirabilis_YR139_2582581314.fna.RAST.mdl']
The minimal flux media for Pantoea_sp._YR343_2511231025.fna.RAST.mdl consists of 7 compounds and a 5.528441881468454e-13 total influx, with a growth value of 8.329773126419129e-15
The minimal flux media for Rhizobium_sp._CF142_2510917030.fna.RAST.mdl consists of 11 compounds and a 5.840632700290867e-15 total influx, with a growth value of 7.659271649448432e-17
{'Pantoea_sp._YR343_2511231025.fna.RAST.mdl': {'media': {'EX_cpd00067_e0': 3.9404282647003083e-13, 'EX_cpd00307_e0': 1.9721488985674257e-29, 'EX_cpd08023_e0': 1.00593597978899e-14, 'EX_cpd03847_e0': 5.56115364133574e-14, 'EX_cpd00128_e0': 2.973281877434766e-14, 'EX_cpd01329_e0': 3.271187946292587e-14, 'EX_cpd15603_e0': 3.068576722829376e-14}, 'solution': <Solution 0.000 at 0x1cba104d0>}, 'Rhizobium_sp._CF142_2510917030

In [5]:
from pandas import read_csv, DataFrame
df = read_csv("/Users/afreiburger/Downloads/main_scores(2).csv")
# dir(df["community growth"])
data = {}
for col in df.columns:
    try:
        # print(col, df[col].min(), df[col].max())
        if any([x in col for x in ["MRO", "BSS"]]):   df[col] = df[col].str.split("%").str[0].astype(float)
        if "MIP" in col:   df[col] = df[col].str.split("(").str[0].astype(float)
        data[col] = {"min": df[col].min(), "max": df[col].max(), "median": df[col].median()}
    except Exception as e:
        print(e, f"The {col} fails to compute the max/min")
# print(df.max(axis=1))
# print(df.min(axis=1))
# print(max(), min())

newD2F = df[df["community growth"] < 2]
data["<2 community growth"] = {"min": newD2F["community growth"].min(), "max": newD2F["community growth"].max(), "count": len(newD2F)}
newDF = df[df["community growth"] > 10]
data[">10 community growth"] = {"min": newDF["community growth"].min(), "max": newDF["community growth"].max(), "count": len(newDF)}

DataFrame(data).T.to_csv("stats.csv")

# newDF.to_csv("filtered_kChip.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/afreiburger/Downloads/main_scores(2).csv'

In [6]:
df

NameError: name 'df' is not defined

In [7]:
print("more than 10x difference: "+str(len(df[df["coculture growth model1"] > df["coculture growth model2"]*10])+len(df[df["coculture growth model1"]*10 < df["coculture growth model2"]]))+f"/{len(df)}")
print("more than 2x difference: "+str(len(df[df["coculture growth model1"] > df["coculture growth model2"]*2])+len(df[df["coculture growth model1"]*2 < df["coculture growth model2"]]))+f"/{len(df)}")
df["growth_ratio"] = df["coculture growth model1"] / df["coculture growth model2"]
df["member1_ratio"] = (df["coculture growth model1"]/df["monoculture growth model1"])
df["member2_ratio"] = (df["coculture growth model2"]/df["monoculture growth model2"])
df["normalized_growth_ratio"] = df["member1_ratio"] / df["member2_ratio"]

# Create the histogram
import matplotlib.pyplot as plt
arry = df["growth_ratio"].to_numpy()
arry[arry < 0.00001] = 1e-3
arry[arry < 1] = 1 / arry[arry < 1]
arry[arry > 10] = 10
# from numpy import log
# log_arry = log(arry)
# log_arry[log_arry < 0.00001] = 0
# log_arry[log_arry > 100] = 100
print(df["growth_ratio"].median())
plt.hist(arry, bins=100, edgecolor='black')
plt.xlabel('relative_growths')
plt.ylabel('Frequency')
plt.title('Relative member growths in coculture')
plt.savefig("relative_growth_distribution.png")
plt.show()

print("more than 10x normalized difference: "+str(len(df[df["normalized_growth_ratio"] > 10])+len(df[df["normalized_growth_ratio"] < 0.1]))+f"/{len(df)}")
print("more than 2x normalized difference: "+str(len(df[df["normalized_growth_ratio"] > 2])+len(df[df["normalized_growth_ratio"] < .5]))+f"/{len(df)}")
arry = df["normalized_growth_ratio"].to_numpy()
arry[arry < 0.00001] = 1e-3
arry[arry < 1] = 1 / arry[arry < 1]
arry[arry > 10] = 10
# log_arry = log(arry)
# log_arry[log_arry < 0.00001] = 0
# log_arry[log_arry > 100] = 100
print(df["normalized_growth_ratio"].median())
plt.hist(arry, bins=100, edgecolor='black')
plt.xlabel('relative_growths')
plt.ylabel('Frequency')
plt.title('Normalized relative member growths in coculture')
plt.savefig("normalized_relative_growth_distribution.png")
plt.show()

arry = df["member2_ratio"].to_numpy()+df["member1_ratio"].to_numpy()
arry[arry < 0.00001] = 1e-3
arry[arry < 1] = 1 / arry[arry < 1]
arry[arry > 10] = 10
# log_arry = log(arry)
# log_arry[log_arry < 0.00001] = 0
# log_arry[log_arry > 100] = 100
print(df["member1_ratio"].median())
print(df["member2_ratio"].median())
plt.hist(arry, bins=100, edgecolor='black')
plt.xlabel('relative_growths')
plt.ylabel('Frequency')
plt.title('Member growth effects from community')
plt.savefig("community_effect_ratio.png")
plt.show()

NameError: name 'df' is not defined

In [109]:
from collections import Counter
from pandas import read_csv
df = read_csv("/Users/afreiburger/Downloads/main_scores(2).csv")
df = df[df["monoculture growth model1"] > 1]
df = df[df["monoculture growth model2"] > 1]
df["model1_abundance"] = df["coculture growth model1"] / df["community growth"]
df["model2_abundance"] = df["coculture growth model2"] / df["community growth"]

print(len(df[df["model1_abundance"] < 0.01]))
print(len(df[df["model2_abundance"] < 0.01]))
print(df["model1_abundance"].min(), df["model1_abundance"].max(), df["model1_abundance"].mean())
print(Counter(df["model1_abundance"].to_list()))
print(df["model2_abundance"].min(), df["model2_abundance"].max(), df["model2_abundance"].mean())
print(Counter(df["model2_abundance"].to_list()))
display(Counter(df["BIT"].to_list()))

10
6
-1.3799010420571257e-14 0.9999250290512427 0.5350835661159317
Counter({0.09198323697178999: 9, 0.09241998631871572: 6, 0.8879404442155096: 6, 0.12711803582643022: 6, 0.9214779589812411: 6, 0.8951230293207603: 6, 0.42086784113674386: 6, 0.38342947346679845: 6, 0.07942053302572245: 6, 0.9187472982843173: 6, 0.9191991633049453: 6, 0.9171273950563112: 6, 0.8952387230022191: 4, 0.8115506985751832: 4, 0.29029669641459216: 4, 0.12721766373265703: 4, 0.11089638632013467: 4, 0.17297547936611846: 4, 0.3534369183797235: 4, 0.182722859103734: 4, 0.8397292399530967: 4, 0.09230320274312395: 4, 0.889194522513903: 4, 0.8275044296033801: 4, 0.7851392321214746: 4, 0.874783025079308: 4, 0.8562229600563437: 4, 0.8268179088976519: 4, 0.8681167077609487: 4, 0.4039202503167748: 4, 0.48711402542259014: 4, 0.18134765625: 4, 0.9024831964152352: 4, 0.7518725135021003: 4, 0.9187492544435167: 3, 0.8724365798300762: 3, 0.8954723801966229: 3, 0.12099904315611276: 3, 0.11291247848751652: 3, 0.09690772792415389: 

Counter({'parasitism': 3422,
         'amensalism': 478,
         'competitive': 190,
         'commensalism': 69,
         'mutualism': 33,
         'neutral': 17,
         nan: 1})

In [110]:
from pandas import read_csv, DataFrame
from numpy import mean, std

interactions = read_csv("/Users/afreiburger/Downloads/interactions.csv")
interactions = interactions[(interactions["Bug 1"] != interactions["Bug 2"])
                            & (interactions["1 on 2: Effect"] != 0)
                            & (interactions["2 on 1: Effect"] != 0)]
interactions["average_effect1"] = (interactions["1 on 2: 75th Percentile"] + interactions["1 on 2: 25th Percentile"]) / 2
interactions["average_effect2"] = (interactions["2 on 1: 75th Percentile"] + interactions["2 on 1: 25th Percentile"]) / 2
interactions["summed"] = interactions["average_effect1"] + interactions["average_effect2"]
classes = list(set(interactions["Class"].to_list()))
display(Counter(interactions["Class"].to_list()))
classes.remove("Inconclusive")
monoculture_yield_growth = read_csv("/Users/afreiburger/Downloads/monoculture_yield_growth.csv")

# display(interactions.loc[404])
# display()

data = []
for cls in classes:
    subset = interactions[interactions["Class"] == cls]
    display(subset)
    print(cls)
    # display(subset.drop([col for col in subset.columns if "average" not in col], axis=1))
    lesser = subset[subset["average_effect1"] < subset["average_effect2"]]
    greater = subset[subset["average_effect1"] > subset["average_effect2"]]
    content = sorted(lesser["average_effect1"].to_list())
    # print(content)
    content2 = sorted(greater["average_effect1"].to_list())
    # content = subset["1 on 2: 75th Percentile"].to_list()
    # display(subset[subset["1 on 2: 75th Percentile"] == content[-1]])
    data.append([f"{max(content):.4f}", f"{min(content):.4f}",  f"{mean(content):.4f}", f"{std(content):.2f}",
                 f"{max(content2):.4f}", f"{min(content2):.4f}",  f"{mean(content2):.4f}", f"{std(content2):.2f}"])

df = DataFrame(data, columns=["max1", "min1", "mean1", "std1", "max2", "min2", "mean2", "std2"],
               index=classes).sort_values(by="mean1", ascending=True)
display(df)

# c12 = interactions[interactions["Class"] == "Commensalism"][interactions["1 on 2: Effect"] < 0]["1 on 2: Effect"]
# com12 = c12.to_list()
# print("commensal", max(com12), min(com12), f"{mean(com12):.4f} +- {std(com12):.2f}")

display(interactions)
# display(monoculture_yield_growth)

Counter({'Competition': 4296,
         'Parasitism': 2244,
         'Amensalism': 1600,
         'Commensalism': 1126,
         'Inconclusive': 824,
         'Neutralism': 622,
         'Mutualism': 536})

,Bug 1,Bug 2,Carbon,1 on 2: Effect,1 on 2: 25th Percentile,1 on 2: 75th Percentile,1 on 2: Replicates,2 on 1: Effect,2 on 1: 25th Percentile,2 on 1: 75th Percentile,2 on 1: Replicates,Class,Type,Strength,average_effect1,average_effect2,summed
23,PAg2,PAg3,ArabinoseD,0.700197,0.059324,1.142397,4,-0.352347,-0.829357,0.189883,10,Commensalism,18.287907,0.783852,0.600860,-0.319737,0.281123
29,PAg2,BI,ArabinoseD,0.967681,0.400615,1.169769,9,-0.390698,-0.834792,0.043895,8,Commensalism,23.013759,1.043577,0.785192,-0.395448,0.389744
61,PAg3,PAg2,ArabinoseD,-0.352347,-0.829357,0.189883,10,0.700197,0.059324,1.142397,4,Commensalism,18.287907,0.783852,-0.319737,0.600860,0.281123
181,BI,PAg2,ArabinoseD,-0.390698,-0.834792,0.043895,8,0.967681,0.400615,1.169769,9,Commensalism,23.013759,1.043577,-0.395448,0.785192,0.389744
413,PAl,EA,ArabinoseL,2.109261,1.663066,2.313951,7,-0.804409,-1.001956,0.104823,6,Commensalism,24.124653,2.257445,1.988509,-0.448567,1.539942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15164,PH,LA,Uridine,-0.001961,-0.342179,1.652504,6,0.852787,0.606238,1.156005,6,Commensalism,44.868215,0.852790,0.655163,0.881122,1.536284
15185,PP,EL,Uridine,1.896594,1.762137,2.060491,9,0.012454,-1.656875,0.742333,12,Commensalism,45.376237,1.896634,1.911314,-0.457271,1.454043
15195,PP,PR1,Uridine,-0.042005,-0.285063,0.175296,12,1.330184,1.261340,1.420830,20,Commensalism,43.191317,1.330847,-0.054883,1.341085,1.286202
15196,PP,PAr,Uridine,0.196373,-0.049348,0.342347,21,0.170987,0.075509,0.284740,19,Commensalism,86.046972,0.260382,0.146499,0.180124,0.326624


Commensalism


,Bug 1,Bug 2,Carbon,1 on 2: Effect,1 on 2: 25th Percentile,1 on 2: 75th Percentile,1 on 2: Replicates,2 on 1: Effect,2 on 1: 25th Percentile,2 on 1: 75th Percentile,2 on 1: Replicates,Class,Type,Strength,average_effect1,average_effect2,summed
25,PAg2,EL,ArabinoseD,0.555210,0.000000,0.657082,8,-2.353877,-2.918099,-1.838397,6,Neutralism,-31.728185,2.418470,0.328541,-2.378248,-2.049707
31,PAg2,CF,ArabinoseD,0.045542,0.000000,0.325028,4,-0.859060,-1.160096,-0.312309,9,Neutralism,-41.965408,0.860266,0.162514,-0.736203,-0.573689
101,EL,PAg2,ArabinoseD,-2.353877,-2.918099,-1.838397,6,0.555210,0.000000,0.657082,8,Neutralism,-31.728185,2.418470,-2.378248,0.328541,-2.049707
114,EL,PK,ArabinoseD,0.166717,0.000000,0.532379,7,0.036565,0.000000,0.085586,9,Neutralism,57.370422,0.170680,0.266189,0.042793,0.308982
214,KA,PK,ArabinoseD,1.243602,1.155278,1.284316,7,0.721651,0.000000,1.716392,4,Neutralism,75.126196,1.437819,1.219797,0.858196,2.077993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15024,CF,LA,Uridine,0.291962,0.030639,0.371492,21,-0.129961,-0.256871,0.000000,13,Neutralism,21.004797,0.319580,0.201065,-0.128435,0.072630
15061,EA,PAg2,Uridine,0.306805,0.000000,0.437625,11,-0.168844,-0.535215,-0.095949,4,Neutralism,16.174717,0.350196,0.218813,-0.315582,-0.096770
15063,EA,PAg3,Uridine,-0.114410,-0.761581,0.000000,11,-0.505828,-0.670865,-0.387954,10,Neutralism,-57.744894,0.518605,-0.380791,-0.529409,-0.910200
15078,EA,PH,Uridine,0.061786,0.000000,0.347494,5,0.131438,0.043640,0.323247,12,Neutralism,70.177140,0.145236,0.173747,0.183444,0.357191


Neutralism


...

Competition


,Bug 1,Bug 2,Carbon,1 on 2: Effect,1 on 2: 25th Percentile,1 on 2: 75th Percentile,1 on 2: Replicates,2 on 1: Effect,2 on 1: 25th Percentile,2 on 1: 75th Percentile,2 on 1: Replicates,Class,Type,Strength,average_effect1,average_effect2,summed
24,PAg2,LA,ArabinoseD,0.970465,0.740470,1.895412,6,-0.590360,-1.010527,-0.159135,10,Parasitism,13.686734,1.135926,1.317941,-0.584831,0.733110
30,PAg2,KA,ArabinoseD,0.904254,0.723154,1.570491,4,-0.622904,-0.991756,-0.174094,11,Parasitism,10.438630,1.098037,1.146822,-0.582925,0.563897
34,PAg2,PK,ArabinoseD,0.250753,0.125376,0.580697,3,-1.000179,-1.871505,-0.545120,6,Parasitism,-30.925592,1.031132,0.353037,-1.208312,-0.855276
81,LA,PAg2,ArabinoseD,-0.590360,-1.010527,-0.159135,10,0.970465,0.740470,1.895412,6,Parasitism,13.686734,1.135926,-0.584831,1.317941,0.733110
201,KA,PAg2,ArabinoseD,-0.622904,-0.991756,-0.174094,11,0.904254,0.723154,1.570491,4,Parasitism,10.438630,1.098037,-0.582925,1.146822,0.563897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15453,EC,EA,Mix,0.497077,0.391934,0.538555,9,-2.112326,-2.162774,-2.029302,13,Parasitism,-31.757953,2.170024,0.465244,-2.096038,-1.630794
15464,EA,LA,Mix,-0.603734,-0.686377,-0.542867,12,0.252906,0.134829,0.335727,16,Parasitism,-22.271004,0.654566,-0.614622,0.235278,-0.379344
15472,EA,EC,Mix,-2.112326,-2.162774,-2.029302,13,0.497077,0.391934,0.538555,9,Parasitism,-31.757953,2.170024,-2.096038,0.465244,-1.630794
15495,PK,PR1,Mix,0.239458,0.099402,0.300839,16,-1.109815,-1.182888,-1.022221,15,Parasitism,-32.824314,1.135354,0.200120,-1.102555,-0.902434


Parasitism


,max1,min1,mean1,std1,max2,min2,mean2,std2
Commensalism,0.7935,-2.2627,-0.0386,0.25,4.7531,0.0969,1.9097,1.05
Neutralism,0.9256,-4.7223,-0.4162,0.67,4.4849,-0.9066,0.3677,0.79
Parasitism,-0.0335,-6.5446,-0.9913,1.07,4.7762,0.0555,1.6998,1.27
Competition,-0.2175,-7.9654,-1.5014,0.83,-0.0524,-2.4912,-0.5879,0.34
Amensalism,-0.0927,-6.1023,-1.6992,1.31,1.8170,-1.1025,-0.0397,0.21
Mutualism,2.9425,0.0349,0.4390,0.46,5.0658,0.2250,2.1452,1.13


,Bug 1,Bug 2,Carbon,1 on 2: Effect,1 on 2: 25th Percentile,1 on 2: 75th Percentile,1 on 2: Replicates,2 on 1: Effect,2 on 1: 25th Percentile,2 on 1: 75th Percentile,2 on 1: Replicates,Class,Type,Strength,average_effect1,average_effect2,summed
23,PAg2,PAg3,ArabinoseD,0.700197,0.059324,1.142397,4,-0.352347,-0.829357,0.189883,10,Commensalism,18.287907,0.783852,0.600860,-0.319737,0.281123
24,PAg2,LA,ArabinoseD,0.970465,0.740470,1.895412,6,-0.590360,-1.010527,-0.159135,10,Parasitism,13.686734,1.135926,1.317941,-0.584831,0.733110
25,PAg2,EL,ArabinoseD,0.555210,0.000000,0.657082,8,-2.353877,-2.918099,-1.838397,6,Neutralism,-31.728185,2.418470,0.328541,-2.378248,-2.049707
26,PAg2,RP2,ArabinoseD,NaN,NaN,NaN,2,-0.234547,-0.603079,0.266930,15,Inconclusive,NaN,NaN,NaN,-0.168074,NaN
27,PAg2,RP1,ArabinoseD,NaN,NaN,NaN,2,0.199137,-0.220512,0.665382,14,Inconclusive,NaN,NaN,NaN,0.222435,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15594,PP,PK,Mix,-0.871275,-0.954151,-0.798298,16,-0.264192,-0.392445,-0.134972,10,Competition,-61.868604,0.910449,-0.876224,-0.263708,-1.139933
15595,PP,PR1,Mix,-0.177962,-0.305836,-0.100159,12,-0.578874,-0.872946,-0.187690,10,Competition,-62.088920,0.605612,-0.202997,-0.530318,-0.733316
15596,PP,PAr,Mix,-0.989710,-1.009870,-0.864074,9,-0.564077,-0.639725,-0.520619,23,Competition,-74.680607,1.139170,-0.936972,-0.580172,-1.517144
15597,PP,PR2,Mix,-0.901310,-1.184342,-0.736779,8,-0.928729,-1.193124,-0.830437,16,Competition,-89.141629,1.294179,-0.960561,-1.011781,-1.972341


# create the MSDB reaction pathway mapping

In [111]:
from modelseedpy.biochem import from_local

msdb = from_local("../../ModelSEEDDatabase")

In [114]:
from json import dump

rxnPathways = {}
for rxn in msdb.reactions:
    display(dir(rxn))
    # rxnPathways[rxn.id]
    break

# pathwayMapping = dict(zip(msdb.reactions))

['__add__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__weakref__',
 '_annotation',
 '_associate_gene',
 '_check_bounds',
 '_dissociate_gene',
 '_genes',
 '_gpr',
 '_id',
 '_lower_bound',
 '_metabolites',
 '_model',
 '_repr_html_',
 '_set_id_with_model',
 '_update_awareness',
 '_upper_bound',
 'abbr',
 'add_metabolites',
 'aliases',
 'annotation',
 'boundary',
 'bounds',
 'build_reaction_from_string',
 'build_reaction_string',
 'check_mass_balance',
 'compartments',
 'compound_ids',
 'copy',
 'delete',
 'delta_g',
 'delta_g_error',
 'ec_num